# 1.正确的实现
>需求：将原始表中的9个独立的sheets（sheets的内容有各个医疗机构列），拆分为按各个医疗机构每个单独一个表（表里有9个sheets）

In [1]:
# Bing
import shutil
import pandas as pd

# 读取原始Excel文件
original_file = '修改1-2023年半年考核各基层医疗机构存在问题 - 副本.xlsx'
# 调整好表头格式的空表.xlsx作为底模板
kongmoban = '空表.xlsx'
xls = pd.ExcelFile(original_file)

# 获取所有工作表的名称
sheet_names = xls.sheet_names

# 从ExcelFile对象里parse提取机构名称列表
danwei = xls.parse(sheet_name=sheet_names[0], usecols=[0])['机构名称'].to_list()
print(danwei)

# 根据机构名称生成需要的ExcelWriter对象列表
excel_writers = [pd.ExcelWriter(f'./result/{i}.xlsx') for i in danwei]
# 将底模板表头拷贝到上述所有对象中
fuxiebiaotou = [shutil.copy(kongmoban, f'./result/{i}.xlsx') for i in danwei]
# 空出对应的表头skip行数
skip_rows = [1, 4, 4, 4, 4, 3, 3, 4, 4]

for sheet_name, skipr in zip(sheet_names, skip_rows):
    df = pd.read_excel(xls, sheet_name=sheet_name, skiprows=skipr, header=None)

    # 创建一个字典，用于存储分割后的数据框
    split_data = {}

    # 根据机构列（第一列）的值将数据分割成多个数据框
    for unit in df[0].unique():
        split_data[unit] = df[df[0] == unit]

    # 将分割后的数据框写入前面准备好的带表头的对应表中
    for i, writer in zip(danwei,fuxiebiaotou):
        # 避免索引越界错误，i不在split_data里，所以使用try
        try:
            # 由于默认engine是xlrd不支持append写入，因此使用openpyxl，...mode='a'...参数
            with pd.ExcelWriter(writer, engine='openpyxl',if_sheet_exists='overlay',mode='a') as w:
                split_data[i].to_excel(w, sheet_name=sheet_name, index=False, startrow=skipr, header=None)
            # 保存分割后的工作簿
                w.save()
        except:
            continue


['埠柳中心卫生院', '斥山街道卫生院', '大疃镇卫生院', '东山街道中心卫生院', '港西镇卫生院', '虎山镇卫生院', '黄山卫生院', '靖海卫生院', '城西街道卫生院', '崂山街道卫生院', '俚岛中心卫生院', '龙须岛卫生院', '马道卫生院', '宁街道卫生院', '人和中心卫生院', '成山中心卫生院', '上庄中心卫生院', '滕家中心卫生院', '王连街道卫生院', '夏庄镇卫生院', '寻山街道卫生院', '荫子镇卫生院', '崖西中心卫生院', '东城区社区卫生服务中心', '西城区社区卫生服务中心', '石岛社区卫生服务中心']


# 2.没有表头的错误实现

In [ ]:
import pandas as pd

# 读取原始Excel文件
original_file = '修改1-2023年半年考核各基层医疗机构存在问题 - 副本.xlsx'
kongmoban = '空表.xlsx'
xls = pd.ExcelFile(original_file)

# 获取所有工作表的名称
sheet_names = xls.sheet_names

# 创建一个新的Excel Writer对象
danwei = xls.parse(sheet_name=sheet_names[0], usecols=[0])['机构名称'].to_list()
print(danwei)

# 创建ExcelWriter对象列表
excel_writers = [pd.ExcelWriter(f'./result/{i}.xlsx') for i in danwei]
fuxiebiaotou = [shutil.copy(kongmoban, f'./result/{i}.xlsx') for i in danwei]
skip_rows = [1, 4, 4, 4, 4, 3, 3, 4, 4]

for sheet_name, skipr in zip(sheet_names, skip_rows):
    df = pd.read_excel(xls, sheet_name=sheet_name, skiprows=skipr, header=None)

    # 创建一个字典，用于存储分割后的数据框
    split_data = {}

    # 根据机构列的值将数据分割成多个数据框
    for unit in df[0].unique():
        split_data[unit] = df[df[0] == unit]

    # 创建一个新的Excel工作簿并将分割后的数据框写入其中
    for i, writer in zip(danwei, excel_writers):
        try:
            split_data[i].to_excel(writer, sheet_name=sheet_name, index=False, startrow=skipr, header=None)
            # 保存分割后的工作簿
            writer.save()
        except:
            continue

# 关闭所有ExcelWriter对象
for writer in excel_writers:
    writer.close()


['埠柳中心卫生院', '斥山街道卫生院', '大疃镇卫生院', '东山街道中心卫生院', '港西镇卫生院', '虎山镇卫生院', '黄山卫生院', '靖海卫生院', '城西街道卫生院', '崂山街道卫生院', '俚岛中心卫生院', '龙须岛卫生院', '马道卫生院', '宁街道卫生院', '人和中心卫生院', '成山中心卫生院', '上庄中心卫生院', '滕家中心卫生院', '王连街道卫生院', '夏庄镇卫生院', '寻山街道卫生院', '荫子镇卫生院', '崖西中心卫生院', '东城区社区卫生服务中心', '西城区社区卫生服务中心', '石岛社区卫生服务中心']
